<a href="https://colab.research.google.com/github/rakesh835/sentiment-analysis-webapp/blob/master/Sentiment_Analysis_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%mkdir ../data
!wget -O ../data/aclImdb_v1.tar.gz http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -zxf ../data/aclImdb_v1.tar.gz -C ../data

--2020-07-23 11:45:29--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘../data/aclImdb_v1.tar.gz’

../data/aclImdb_v1. 100%[===================>]  80.23M  69.8MB/s    in 1.1s    

2020-07-23 11:45:31 (69.8 MB/s) - ‘../data/aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [ ]:
import os
import glob

def read_imdb_data(data_dir='../data/aclImdb'):
    data = {}
    labels = {}
    
    for data_type in ['train', 'test']:
        data[data_type] = {}
        labels[data_type] = {}
        
        for sentiment in ['pos', 'neg']:
            data[data_type][sentiment] = []
            labels[data_type][sentiment] = []
            
            path = os.path.join(data_dir, data_type, sentiment, '*.txt')
            files = glob.glob(path)
            
            for f in files:
                with open(f) as review:
                    data[data_type][sentiment].append(review.read())
                    # Here we represent a positive review by '1' and a negative review by '0'
                    labels[data_type][sentiment].append(1 if sentiment == 'pos' else 0)
                    
            assert len(data[data_type][sentiment]) == len(labels[data_type][sentiment]), \
                    "{}/{} data size does not match labels size".format(data_type, sentiment)
                
    return data, labels

In [ ]:
data, labels = read_imdb_data()
print("IMDB reviews: train = {} pos / {} neg, test = {} pos / {} neg".format(
            len(data['train']['pos']), len(data['train']['neg']),
            len(data['test']['pos']), len(data['test']['neg'])))

IMDB reviews: train = 12500 pos / 12500 neg, test = 12500 pos / 12500 neg


In [ ]:
from sklearn.utils import shuffle

def prepare_imdb_data(data, labels):
    """Prepare training and test sets from IMDb movie reviews."""
    
    #Combine positive and negative reviews and labels
    data_train = data['train']['pos'] + data['train']['neg']
    data_test = data['test']['pos'] + data['test']['neg']
    labels_train = labels['train']['pos'] + labels['train']['neg']
    labels_test = labels['test']['pos'] + labels['test']['neg']
    
    #Shuffle reviews and corresponding labels within training and test sets
    data_train, labels_train = shuffle(data_train, labels_train)
    data_test, labels_test = shuffle(data_test, labels_test)
    
    # Return a unified training data, test data, training labels, test labets
    return data_train, data_test, labels_train, labels_test

In [ ]:
train_X, test_X, train_y, test_y = prepare_imdb_data(data, labels)
print("IMDb reviews (combined): train = {}, test = {}".format(len(train_X), len(test_X)))

IMDb reviews (combined): train = 25000, test = 25000


In [ ]:
print(train_X[100])
print(train_y[100])

It's a shame that Deliverance is mainly known as the redneck rape movie and for Dueling Banjos. Even people that have seen the film can't get their mind off of that rape scene. It's not as bad as the rape scene in Pulp Fiction. It's certainly not as bad as any female rape scene in just about any movie. People tend to miss the power of the film that contains the infamous buggery scene.<br /><br />The acting, plot, cinematography, and soundtrack of Deliverance all lend a hand to it's brooding charisma. The backcountry it was shot in is beautiful and is quite in contrast to the dark subject matter. The actors both major and minor make you feel like you are rafting down that river right along with them. <br /><br />The thing that separates this film from others is the tangible sense of dread that it inspires. Not many films can make you feel this creeped out. Bottom Line: This movie is a classic. I can't really say much more than that.
1


In [ ]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stoplist = stopwords.words('english')

import re
from bs4 import BeautifulSoup

from nltk.stem.porter import *
stemmer = PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def preprocessing(data_for_preprocess):
  data=[]
  for review in data_for_preprocess:
    string=""
    
    text = BeautifulSoup(review, "html.parser").get_text()
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
  
    for word in text.lower().split():
      if word not in stoplist: # removing stopwords from reviews
        word=PorterStemmer().stem(word)
        string=string+word+" "
        
    data.append(string.rstrip())
    
  return data

In [ ]:
training_data=preprocessing(train_X)

In [ ]:
training_data[2:5]

['movi badli written direct act beggar belief remad better script director cast servic worst problem act jennif beal one hand polish profession total believ hand ri chard woefulli miscast jar particular piec peter gallagh jenni levin aw slave own keep coupl although normal fine work actor director attempt accent inconsist unbeliev much better concentr good job actual english cast ludicr children african merchant thu less social desir gen de couleur societi cast pale skin actor supposedli social desir marcel pronounc african featur includ obvious dy blond fro cast director cannot bother read script cast chose appropri actor larg pool extrem talent physic divers actor color weird could great movi made peopl respect materi choos appropri skill actor plenti good actor would fun see jennif beal daniel sunjata gloria reuben would appropri cast good script decent direct',
 'admit holli watch list edinburgh film festiv howev artist director festiv specif recommend film audienc 200 peopl prior 

In [ ]:
import numpy as np

training_labels=np.array(train_y)

In [ ]:
training_labels[2:5]

array([0, 1, 0])

In [ ]:
max_length=[len(i.split()) for i in training_data]
print(max(max_length))

1429


In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

oov_tok="<OOV_tok>"
padding="post"
max_length=200
trunc_type="post"
vocab_size=30000

#using tokenization on train data
tokenizer=Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_data)
word_index=tokenizer.word_index

train_sequence=tokenizer.texts_to_sequences(training_data)
train_pad_sequence=pad_sequences(train_sequence, padding=padding, maxlen=max_length, truncating=trunc_type)

In [ ]:
import pickle

with open('tokenizer.pickle', 'wb') as handle:
  pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
reverse_word_index=dict([(j, i) for i, j in word_index.items()])

In [ ]:
len(reverse_word_index)

51858

In [ ]:
def decode_review(review):
    return " ".join([reverse_word_index.get(i, "?") for i in review])

print(decode_review(train_pad_sequence[1]))
print(train_X[1:2])

two sailor leav ladi man joseph bradi gene kelli shi innoc clarenc doolittl frank sinatra meet beauti susan abbott kathryn grayson fall love probabl guess stori even 1940 music ridicul everyth nice wholesom get annoy pretti quick also movi far long 140 minut way much silli stori also bore number jose iturbi orchestra still worth catch kelli danc sinatra grayson sing becom magic none song particularli memor sinatra beauti voic care shot rich technicolor gloss mgm act ok kelli fine although see ladi man push sinatra great although see shi guy push grayson given noth incred beauti look shot liter took breath away plenti highlight sinatra kelli big danc sing number sinatra sing anyth grayson two song justli famou anim sequenc kelli danc jerri anim mous tom funni cameo also littl dean stockwel steal everi scene long plot hold still worth catch huge hit day ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?
["Two sailors are on leave

In [ ]:
embid_dim=16

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embid_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 16)           480000    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 200, 128)          41472     
_________________________________________________________________
bidirectional_3 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 566,913
Trainable params: 566,913
Non-trainable params: 0
_________________________________________________________________


In [ ]:
epochs=6

history=model.fit(train_pad_sequence, training_labels, epochs=epochs)

Epoch 1/6
782/782 [==============================] - 27s 35ms/step - loss: 0.4400 - accuracy: 0.7960
Epoch 2/6
782/782 [==============================] - 27s 34ms/step - loss: 0.2316 - accuracy: 0.9150
Epoch 3/6
782/782 [==============================] - 27s 34ms/step - loss: 0.1447 - accuracy: 0.9502
Epoch 4/6
782/782 [==============================] - 27s 34ms/step - loss: 0.0956 - accuracy: 0.9696
Epoch 5/6
782/782 [==============================] - 27s 34ms/step - loss: 0.0573 - accuracy: 0.9816
Epoch 6/6
782/782 [==============================] - 27s 34ms/step - loss: 0.0679 - accuracy: 0.9772


In [ ]:
test_sequence=tokenizer.texts_to_sequences(test_X)
test_pad_sequence=pad_sequences(test_sequence, maxlen=max_length, padding=padding, truncating=trunc_type)

In [ ]:
predictions=model.predict(test_pad_sequence)

In [ ]:
test_labels=np.array(test_y)

In [ ]:
len(test_labels)

25000

In [ ]:
predict=[]
for value in predictions:
  if value>=0.5:
    predict.append(1)
  else:
    predict.append(0)

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(test_labels, predict))

0.6298


In [ ]:
predict[:10]

[1, 1, 1, 0, 0, 0, 0, 0, 1, 0]

In [ ]:
test_labels[:10]

array([0, 1, 1, 0, 0, 1, 0, 0, 1, 1])

In [ ]:
review="i don't like lsd and ggm as it's too long, very bad movie don't waste your time"

In [ ]:
review=review.split(' ',0)
test_sequence=tokenizer.texts_to_sequences(review)
testing=pad_sequences(test_sequence, maxlen=max_length, padding=padding, truncating=trunc_type)

In [ ]:
output=model.predict(testing)

In [ ]:
output

array([[0.17960969]], dtype=float32)

In [ ]:
if output[0][0]>=0.5:
  print('+ve')
else:
  print('-ve')

-ve


In [ ]:
import time

export_path='./sentiment_analysis{}.h5'.format(int(time.time()))
model.save(export_path)